In [77]:
import pandas as pd
from sqlalchemy import create_engine
import requests
# import json
import os
import urllib, json

In [78]:
csv_file = "Resources/nyt_table.csv"
nyt_file = pd.read_csv(csv_file)
nyt_file.head()

,STATE,CASES DAILY AVG,"PER ""100,000""",14 DAY CHANGE,HOSPITALIZED DAILY AVG,"PER ""100,000"".1",14 DAY CHANGE.1,DEATHS DAILY AVG,"PER ""100,000"".2",FULLY VACCINATED
0,United States,"148,202",45,–8%,"94,101",28,–8%,"2,010.70",0.61,54%
1,Tennessee ›,"7,438",109,9%,"3,323",49,–2%,63.9,0.94,44%
2,West Virginia ›,"1,833",102,23%,972,54,31%,23.3,1.3,40%
3,Alaska ›,738,101,33%,214,29,23%,1.6,0.21,49%
4,Kentucky ›,"4,362",98,Flat,"2,546",57,5%,37.3,0.83,51%


In [79]:
nyt_file_df = nyt_file[[ 'STATE', 'CASES DAILY AVG', 'HOSPITALIZED DAILY AVG', 'DEATHS DAILY AVG', 'FULLY VACCINATED']]
nyt_file_df.head()

,STATE,CASES DAILY AVG,HOSPITALIZED DAILY AVG,DEATHS DAILY AVG,FULLY VACCINATED
0,United States,"148,202","94,101","2,010.70",54%
1,Tennessee ›,"7,438","3,323",63.9,44%
2,West Virginia ›,"1,833",972,23.3,40%
3,Alaska ›,738,214,1.6,49%
4,Kentucky ›,"4,362","2,546",37.3,51%


In [80]:

# covidactnow.org



csv_file = "Resources/states.csv"
states_file = pd.read_csv(csv_file)
states_file.head()


,fips,country,state,county,level,lat,locationId,long,population,metrics.testPositivityRatio,...,actuals.vaccinesDistributed,actuals.vaccinationsInitiated,actuals.vaccinationsCompleted,lastUpdatedDate,url,metrics.vaccinationsInitiatedRatio,metrics.vaccinationsCompletedRatio,actuals.newDeaths,actuals.vaccinesAdministered,cdcTransmissionLevel
0,2,US,AK,NaN,state,NaN,iso1:us#iso2:us-ak,NaN,731545,0.104,...,1024575,411978,356784,9/18/2021,https://covidactnow.org/us/alaska-ak,0.563,0.488,1,763376,3
1,1,US,AL,NaN,state,NaN,iso1:us#iso2:us-al,NaN,4903185,0.175,...,6525240,2535635,2010919,9/18/2021,https://covidactnow.org/us/alabama-al,0.517,0.410,192,4383430,3
2,5,US,AR,NaN,state,NaN,iso1:us#iso2:us-ar,NaN,3017804,0.099,...,3925390,1656649,1336396,9/18/2021,https://covidactnow.org/us/arkansas-ar,0.549,0.443,18,2938812,3
3,4,US,AZ,NaN,state,NaN,iso1:us#iso2:us-az,NaN,7278717,0.110,...,9711660,4252340,3647534,9/18/2021,https://covidactnow.org/us/arizona-az,0.584,0.501,19,8045806,3
4,6,US,CA,NaN,state,NaN,iso1:us#iso2:us-ca,NaN,39512223,0.048,...,57390915,27860963,22769456,9/18/2021,https://covidactnow.org/us/california-ca,0.705,0.576,123,49458723,3


In [81]:
states_file.columns

Index(['fips', 'country', 'state', 'county', 'level', 'lat', 'locationId',
       'long', 'population', 'metrics.testPositivityRatio',
       'metrics.testPositivityRatioDetails.source', 'metrics.caseDensity',
       'metrics.contactTracerCapacityRatio', 'metrics.infectionRate',
       'metrics.infectionRateCI90', 'metrics.icuHeadroomRatio',
       'metrics.icuHeadroomDetails.currentIcuCovid',
       'metrics.icuHeadroomDetails.currentIcuCovidMethod',
       'metrics.icuHeadroomDetails.currentIcuNonCovid',
       'metrics.icuHeadroomDetails.currentIcuNonCovidMethod',
       'metrics.icuCapacityRatio', 'riskLevels.overall',
       'riskLevels.testPositivityRatio', 'riskLevels.caseDensity',
       'riskLevels.contactTracerCapacityRatio', 'riskLevels.infectionRate',
       'riskLevels.icuHeadroomRatio', 'riskLevels.icuCapacityRatio',
       'actuals.cases', 'actuals.deaths', 'actuals.positiveTests',
       'actuals.negativeTests', 'actuals.contactTracers',
       'actuals.hospitalBeds.cap

In [82]:
states_reduced1_df = states_df[['country', 'state', "actuals.hospitalBeds.currentUsageCovid", "actuals.icuBeds.currentUsageCovid", "actuals.vaccinationsCompleted"]]
states_reduced1_df.head()

,country,state,actuals.hospitalBeds.currentUsageCovid,actuals.icuBeds.currentUsageCovid,actuals.vaccinationsCompleted
0,US,AK,217.0,49.0,356784
1,US,AL,2445.0,744.0,2010919
2,US,AR,1086.0,382.0,1336396
3,US,AZ,2040.0,530.0,3647534
4,US,CA,6846.0,1808.0,22769456


In [83]:
states_rename_df = states_reduced1_df.rename(columns={"actuals.hospitalBeds.currentUsageCovid":"Hospital_Beds-Covid",
                                                         "actuals.icuBeds.currentUsageCovid":"ICU_Beds-Covid",
                                                         "actuals.vaccinationsCompleted":"Vaccinations_Completed"})
states_rename_df.head()

,country,state,Hospital_Beds-Covid,ICU_Beds-Covid,Vaccinations_Completed
0,US,AK,217.0,49.0,356784
1,US,AL,2445.0,744.0,2010919
2,US,AR,1086.0,382.0,1336396
3,US,AZ,2040.0,530.0,3647534
4,US,CA,6846.0,1808.0,22769456


In [84]:
# url = "https://api.covidactnow.org/v2/states.json?apiKey=b81407aad07647a492c9853c6800001c"
# url
# 

In [85]:
# print(requests.get(url))

In [86]:
# print(requests.get(url).json())

In [87]:
# response = requests.get(url).json()
# print (json.dumps(response, indent=4, sort_keys=True))

In [88]:
# FIELDS = ['list of keys I care about']
# def clean_data(data):
#     table = pd.DataFrame()
#     for i in range(len(data) - 1):
#         df = pd.json_normalize(data[i + 1])
#         df = df[FIELDS]
#         table = table.append(df)
#     return table




In [89]:
# df.columns


In [90]:
# df.head()

In [91]:
# filepath  = os.path.join("Resources/hospitalizations.json")
# with open (filepath) as jsonfile:
#    json_data = json.load(jsonfile)
#     print(json_data)

In [99]:
# https://www.kff.org/    ttps://covid.ourworldindata
    
    
data = json.load(open('Resources/hospitalizations.json'))

df = pd.DataFrame(data["data"])

# patients_df = pd.read_json('E:/datasets/patients.json')
# patients_df.head()

# import json
# import pandas as pd
# data = json.load(open('json_file.json'))

# df = pd.DataFrame(data["result"])

df.columns


Index(['FIPS', 'StateAbbreviation', 'StateName', 'Date', 'StateReportedDate',
       'TotalHospToDate', 'TotalInICUToDate', 'CurrentHospitalizations',
       'CurrentlyInICU', 'CurrentlyOnVentilator', 'TotalDeaths'],
      dtype='object')